In [5]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [31]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
dfs = pd.read_html(url)
hoods = dfs[10]
hoods.drop(['Neighbourhoods covered','Map', 'Unnamed: 5'], axis = 1)

,CDN number,City-designated neighbourhood,Former city/borough
0,129,Agincourt North,Scarborough
1,128,Agincourt South-Malvern West,Scarborough
2,20,Alderwood,Etobicoke
3,95,Annex,Old City of Toronto
4,42,Banbury-Don Mills,North York
...,...,...,...
135,94,Wychwood,Old City of Toronto
136,100,Yonge and Eglinton,Old City of Toronto
137,97,Yonge-St.Clair,Old City of Toronto
138,27,York University Heights,North York
